- 本节将以带**L2范数正则化**和含**单隐藏层**的多层感知机为样例模型解释正向传播和反向传播

## 3.14.1 正向传播
- 正向传播是指对神经网络沿着输入层到输出层的顺序，依次计算并存储模型的中间变量。为简单起见，假设输入是一个特征为$x\in{R^d}$的样本，且不考虑偏差项，那么中间变量
$$
    z = W^{(i)}x
$$
其中$W^{(i)} \in {R^{h×d}}$是隐藏层的权重参数。把中间变量$z \in {R^h}$输入按元素运算的激活函数$\phi$后，将的到向量长度为h的隐藏层变量
$$
    h = \phi(z)
$$
隐藏层变量h也是一个中间变量。假设输出层参数只有权重$W^{(2)}\in{R{q×h}}$，可以的到向量长度为q的输出曾变量
$$
    o = W^{(2)}h
$$
假设损失函数为$l$，且样本标签为y，可以计算出单个数据样本的损失项
$$
    L = l(o,y)
$$
根据$L_2$范数正则化的定义，给定超参数$\lambda$，正则化项即
$$
    s = \frac{\lambda}{2}(\parallel{W^{(1)}}\parallel^2_F + \parallel{W^{(2)}}\parallel^2_F)
$$
其中矩阵的Frobenius范数等价于将矩阵变平为向量后计算$L_2$范数。最终，模型在给定的数据样本上带正则化的损失为
$$
    J = L + s
$$
我们将J称为有关给定数据样本的目标函数，并在以下的讨论中简称目标函数

### 3.14.2 正向传播的计算图
![1](../image/3.14_1.png)

### 3.14.3 反向传播
- 反向传播指的是**计算神经网络参数梯度**的方法。总的来说，反响传播依据微积分中的链式法则，沿着从输出层到输入层的顺序，依次计算并存储目标函数有关神经网络各层的中间变量以及参数的梯度。对输入或输出X，Y，Z为任意形状张量的函数Y=f(x)和Z=g(Y)，通过链式法则，我们有
$$
    \frac{\partial{Z}}{\partial{X}} = prod(\frac{\partial{Z}}{\partial{Y}},\frac{\partial{Y}}{\partial{X}})
$$
其中prod运算符将根据连个输入的形状，在必要的操作（如转置和互换输入位置）后对两个输入做乘法。
回顾一下本节中的样例模型，它的参数是$W^{(1)}$和$W^{(2)}$，因此反向传播的目标是计算$\partial{J}/\partial{W^{(1)}}$和$\partial{J}/\partial{W^{(2)}}$。我们将应用链式法则依次计算各中间变量和参数的梯度，其计算次序与前向传播中相应中间变量的计算次序恰恰相反。首先，分别计算目标函数J = L + s有关损失项L和正则项s的梯度
$$
    \frac{\partial{J}}{\partial{L}} = 1, 
    \frac{\partial{J}}{\partial{s}} = 1
$$
其次，依据链式法则计算目标函数有关输出层变量的梯度$\partial{J}/\partial{o}\in{R^q}$(J = L + s,L = loss(o,y)):
$$
    \frac{\partial{J}}{\partial{o}} = prod(\frac{\partial{J}}{\partial{L}},\frac{\partial{L}}{\partial{o}}) = \frac{\partial{L}}{\partial{o}}
$$
接下来，计算正则项有关两个参数的梯度($s = \parallel{W}\parallel^2_2$)：
$$
    \frac{\partial{s}}{\partial{W^{(1)}}} = \lambda{W^{(1)}},
    \frac{\partial{s}}{\partial{W^{(2)}}} = \lambda{W^{(2)}}.
$$
现在，我们可以计算最靠近输出层的模型参数的梯度${\partial{J}}{\partial{W^{(2)}}}\in{R^{(q×h)}}$.依据链式法则，得到
$$
    {\partial{J}}{\partial{W^{(2)}}} = prod(\frac{\partial{J}}{\partial{o}},\frac{\partial{o}}{\partial{W^{(2)}}}) + prod(\frac{\partial{J}}{\partial{s}},\frac{\partial{s}}{\partial{W^{(2)}}}) = \frac{\partial{J}}{\partial{o}}\mathbf{h}^\mathrm{T} + \lambda{W^{(2)}}
$$
沿着输出层向隐藏层继续反向传播，隐藏层变量的梯度${\partial{J}}{\partial{h}}\in{R^{h}}$可以这样计算：
$$
    \frac{\partial{J}}{\partial{h}} = prod(\frac{\partial{J}}{\partial{o}},\frac{\partial{o}}{\partial{h}}) = \mathbf{W^{(2)}}^\mathrm{T}\frac{\partial{J}}{\partial{o}}
$$.
由于激活函数$\phi$是按元素运算的，中间变量z的梯度${\partial{J}}{\partial{z}}\in{R^{(h)}}$的计算需要使用按元素乘法符$\odot$：
$$
    \frac{\partial{J}}{\partial{z}} = prod(\frac{\partial{J}}{\partial{h}},\frac{\partial{h}}{\partial{z}}) = \frac{\partial{J}}{\partial{h}}\odot{\phi^{'}(z)}.
$$
最终，我们可以得到最靠近输入层的模型参数的梯度${\partial{J}}{\partial{W^{(1)}}}$。依据链式法则，得到
$$
     \frac{\partial{J}}{\partial{W^{(1)}}} = prod( \frac{\partial{J}}{\partial{z}}, \frac{\partial{z}}{\partial{W^{(1)}}}) + prod( \frac{\partial{J}}{\partial{s}},  \frac{\partial{s}}{\partial{W^{(1)}}}) =  \frac{\partial{J}}{\partial{z}}\mathbf{x}^\mathrm{T} + \lambda{W^{(1)}}
$$

### 总结反向传播
- 反向传播的本质就是，通过链式求导法则，求出参数的梯度，关键是要理清函数变量直接的关系

## 3.14.3 训练深度学习模型
- 正向传播和反向传播互相依赖
    - 一方面，正向传播的计算可能依赖于模型参数的当前值(要靠当前值往后算)，而这些参数模型是在反向传播的梯度计算后通过优化算法迭代的。如$s = (\lambda/2)(\parallel{W^{(1)}}\parallel^2_F + \parallel{W^{(2)}}\parallel^2_F)$依赖模型参数$W^{(1)}$和$W^{(2)}$的当前值，而这些当前值是优化算法最近一次根据反向传播算出梯度后迭代得到的。（总结一下就是正向传播使用反向传播得到的优化参数继续迭代，不断循环）
    - 另一方面，反向传播的梯度计算可能依赖于各变量的当前值，而这些变量的当前值是通过正向传播得到的。（反正就是你中有我，我中有你，不可分割你懂吧）
    - **我们在训练模型时是正向传播和反向传播交替进行，正向传播得到结果，与label比对，反向传播优化参数，较小损失值，把优化后的参数在正向传播使用，如此循环往复**

### 小结
- 正向传播沿着从输入层到输出层的顺序，依次计算并存储神经网络的中间变量
- 反向传播沿着从输出层到输入层的顺序，依次计算并存储神经网络中间变量和参数的梯度
- 在训练深度学习模型时，正向传播和反向传播相互依赖